In [38]:
import numpy as np
from src.backend.state_estimation.config.vehicle_params import VehicleParams
from src.backend.state_estimation.measurments.measurement_transformation.steering_to_wheel_angle import measure_delta_wheel_angle

In [39]:
def moment_coefficients(steering: float):
    delta = measure_delta_wheel_angle(steering)
    lf = VehicleParams.lf
    a2 = VehicleParams.a / 2
    b2 = VehicleParams.b / 2
    return np.array([
        lf * np.sin(delta[0]) + a2 * np.cos(delta[0]), lf * np.sin(delta[1]) - a2 * np.cos(delta[1]), b2, -b2,
    ])

In [40]:
Tmax = np.array([50, 50, 200, 200])
Tcmd = 300 
dT = 200

In [41]:
A_raw = np.array([moment_coefficients(0)], dtype=np.float64)
B_raw = np.linalg.pinv(A_raw)
print(A_raw)
print(B_raw)
torques = B_raw @ np.array([dT], dtype=np.float64)
print('Hello')
print(torques)

[[ 0.62051896 -0.62051896  0.62       -0.62      ]]
[[ 0.40322567]
 [-0.40322567]
 [ 0.40288843]
 [-0.40288843]]
Hello
[ 80.64513306 -80.64513306  80.57768646 -80.57768646]


In [42]:
Tmax = np.array([50, 50, 200, 200])
Tcmd = 300 
dT = 200
A_raw = np.array([[1, 1, 1, 1], [1, -1, 1, -1]])
A_norm = A_raw * Tmax 
goal = np.array([Tcmd, dT], dtype=np.float64)

In [43]:
B = np.linalg.pinv(A_norm)
print(A_norm)
print(B)
res_norm = B @ goal
res = res_norm * Tmax
print(res_norm, sum(res_norm))
print(res, sum(res))
print(A_norm @ res_norm)
print(A_raw @ res)

[[  50   50  200  200]
 [  50  -50  200 -200]]
[[ 0.00058824  0.00058824]
 [ 0.00058824 -0.00058824]
 [ 0.00235294  0.00235294]
 [ 0.00235294 -0.00235294]]
[0.29411765 0.05882353 1.17647059 0.23529412] 1.7647058823529416
[ 14.70588235   2.94117647 235.29411765  47.05882353] 300.0
[300. 200.]
[300. 200.]


In [44]:
res_soft_max = np.exp(res_norm) / np.sum(np.exp(res_norm))
print(res_soft_max, sum(res_soft_max))

[0.19418265 0.15347    0.46925774 0.18308961] 1.0
